In [76]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import string                              
from nltk.corpus import stopwords          
from nltk.stem import PorterStemmer        
from nltk.tokenize import TweetTokenizer
from IPython.display import clear_output
from collections import defaultdict
import numpy as np

def etiquetas(palabras,frecuencias,clases):
    x = np.zeros((1, 6))
    for palabra in palabras:
        for t in clases:
            x[0,t] += frecuencias.get((palabra, t),0)
    x[0,0]=1
    return x

def limpiar(st):
    if st == "" :return ""
    res = []
    st = st.replace("(","").replace(")","").replace(",","").replace(".","")
    for t in range(100):
        st = st.replace("[" + str(t) + "]", "")
    for letra in ["a","b","c"]:
        st = st.replace("[" + letra + "]", "")
    st = st.split()
    for i in range(len(st)-1):
        if st[i].isdecimal() and ( st[i+1].lower() == "ad"  or st[i+1].lower() == "bc"  or 
                                   st[i+1].lower() == "ac"  or st[i+1].lower() == "dc"):
            res.append("".join([st[i],st[i+1]]))
        elif st[i].lower() not in {"ad","bc","ac","dc"}:
            res.append(st[i])
    if st[-1].lower() not in {"ad","bc","ac","dc"}: 
        res.append(st[-1])
    return " ".join(res)

stop        = stopwords.words('english')
tokenizer   = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
stemmer     = PorterStemmer()
webs        = pd.ExcelFile("WEBs2.xls")
df          = webs.parse("Categorias")
textos2     = pd.DataFrame(columns = [ 'Texto' , 'Clase'])
clases_df   = set(df["CÓDIGO"].value_counts().index)
frecuencias = defaultdict(int)
for i in df.index:
    url       = df["URL"][i]
    pagina    = get(url)
    html      = BeautifulSoup(pagina.text,"html.parser")
    parrafos  = [x.text.strip() for x in html.findAll("p") if x.text != "\n"  ]
    for p in parrafos:
        palabras = limpiar(p)
        if 15 <= len(palabras.split()) < 150:
            texto_tokens = tokenizer.tokenize(palabras)
            texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation ]
            texto_stem   = [ stemmer.stem(word) for word in texto_clean ]
            
            #for t in range(len(texto_stem)//20):
            #    j = 20 * t
            #    p  = texto_stem[ j : 20 * (t+1) ]
            #    textos2   = textos2.append({'Texto' : p, 'Clase' : df["CÓDIGO"][i] }, ignore_index = True)
            
            textos2   = textos2.append({'Texto' : texto_stem, 'Clase' : df["CÓDIGO"][i] }, ignore_index = True)
    clear_output( wait = True )
    print("procesando {}".format(url))
    print(f"{i+1} de {len(df)}")

vocabulario = set([x for m in textos2.index for x in textos2.loc[m].Texto ])                
clases_df   = set(df["CÓDIGO"].value_counts().index)

for i in textos2.index:
    for palabra in textos2.loc[i].Texto:
        frecuencias[(palabra, textos2.loc[i].Clase)] += 1
    clear_output( wait = True )
    print(f"Creaando Frecuencias {i+1} de {len(textos2)}")

for i in clases_df:
    textos2[f"Clase{i}"] = textos2.Texto.apply( lambda x : etiquetas(x,frecuencias,clases_df)[0,i])

textos2["FreqOK"]=None
for i in textos2.index:
    textos2["FreqOK"][i] = int(np.argmax(textos2.iloc[i,-6:-1])) + 1 == int(textos2.iloc[i,1:2])

print("dataset terminado")
print(textos2.describe())


Creaando Frecuencias 4924 de 4924


/home/emi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


dataset terminado
            Clase1       Clase2       Clase3       Clase4        Clase5
count  4924.000000  4924.000000  4924.000000  4924.000000   4924.000000
mean   1611.689886  1707.607433  1529.614338  1577.364947   1656.729082
std     940.242327   967.284145   849.721900  1170.772442   1383.575976
min      30.000000    59.000000    49.000000    30.000000      9.000000
25%     951.000000  1008.000000   901.750000   776.000000    765.000000
50%    1453.500000  1557.000000  1403.500000  1280.000000   1273.000000
75%    2078.500000  2225.000000  2014.250000  2037.500000   2064.250000
max    8036.000000  8741.000000  5947.000000  9968.000000  11614.000000


In [77]:
textos2

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5,FreqOK
0,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",1,4173.0,3405.0,2836.0,2097.0,2517.0,True
1,"[youth, alexand, tutor, aristotl, age, 16, phi...",1,4124.0,3734.0,2444.0,2202.0,1882.0,True
2,"[alexand, endeavour, reach, end, world, great,...",1,2897.0,2683.0,2494.0,3152.0,1494.0,False
3,"[alexand, born, pella, capit, kingdom, macedon...",1,2297.0,1684.0,1719.0,816.0,873.0,True
4,"[sever, legend, surround, alexander', birth, c...",1,2021.0,862.0,1178.0,515.0,1019.0,True
...,...,...,...,...,...,...,...,...
4919,"[mani, hunter-gather, conscious, manipul, land...",5,987.0,857.0,834.0,673.0,1830.0,True
4920,"[agriculturalist, also, regularli, hunt, gathe...",5,1511.0,1482.0,1491.0,1359.0,2410.0,True
4921,"[evid, suggest, big-gam, hunter-gather, cross,...",5,1579.0,1477.0,1690.0,1101.0,3904.0,True
4922,"[hunter-gather, would, eventu, flourish, ameri...",5,1875.0,1835.0,1794.0,1917.0,3933.0,True


In [78]:
print(textos2.loc[0].Texto)

['alexand', 'iii', 'macedon', 'greek', 'αλέξανδρος', 'γʹ', 'ὁ', 'μακεδών', 'aléxandro', 'iii', 'ho', 'makedȏn', '20/21', 'juli', '356bc', '–', '10/11', 'june', '323bc', 'commonli', 'known', 'alexand', 'great', 'greek', 'ὁ', 'μέγας', 'ho', 'méga', 'king', 'basileu', 'ancient', 'greek', 'kingdom', 'macedon', 'member', 'argead', 'dynasti', 'born', 'pella', '356bc', 'succeed', 'father', 'philip', 'ii', 'throne', 'age', '20', 'spent', 'rule', 'year', 'unpreced', 'militari', 'campaign', 'western', 'asia', 'northeast', 'africa', 'age', 'thirti', 'creat', 'one', 'largest', 'empir', 'ancient', 'world', 'stretch', 'greec', 'northwestern', 'india', 'undef', 'battl', 'wide', 'consid', 'one', "history'", 'success', 'militari', 'command']


In [79]:
freqc = defaultdict(dict)
lista_freq = [[t[0][0],t[0][1],t[1]] for t in sorted(frecuencias.items(),
                                                     key = lambda x: x[0][0])]
for i in lista_freq:
    freqc [f"Clase{i[1]}"] [i[0]] = i[2]
palabras_en_clases = pd.DataFrame(freqc,columns = sorted (freqc.keys())).fillna(0)

In [82]:
palabras_en_clases[(palabras_en_clases.Clase1 > 20) & (palabras_en_clases.Clase2 > 20) &
                   (palabras_en_clases.Clase3 > 20) & (palabras_en_clases.Clase4 > 20) &
                   (palabras_en_clases.Clase5 > 20)].sample(15)

,Clase1,Clase2,Clase3,Clase4,Clase5
state,63.0,142.0,98.0,264.0,27.0
relat,23.0,30.0,26.0,30.0,47.0
term,36.0,80.0,47.0,39.0,74.0
new,110.0,113.0,152.0,160.0,85.0
back,49.0,29.0,35.0,48.0,40.0
would,115.0,111.0,133.0,139.0,78.0
chang,29.0,34.0,45.0,46.0,89.0
form,81.0,68.0,25.0,49.0,73.0
northern,29.0,64.0,44.0,24.0,72.0
caus,34.0,25.0,52.0,45.0,49.0


In [90]:
palabras_en_clases.loc[["roman","roman-parthian"]]

,Clase1,Clase2,Clase3,Clase4,Clase5
roman,546.0,182.0,79.0,16.0,4.0
roman-parthian,1.0,0.0,0.0,0.0,0.0
